In [0]:
# Utill fns for bigquery
from os import path

from google.cloud import bigquery
from google.oauth2 import service_account
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()
from sklearn import preprocessing
#from scipy.stats import shapiro
from scipy.cluster.hierarchy import dendrogram, linkage
from scipy.cluster.hierarchy import fcluster
from sklearn.cluster import AgglomerativeClustering
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn import metrics
from scipy.spatial.distance import cdist
from google.cloud import bigquery
from google.oauth2 import service_account
import sys
from os import path

In [2]:
from google.colab import auth
auth.authenticate_user()
print('Authenticated')

Authenticated


In [0]:
project_id = 'nimble-valve-257105'

In [0]:
#import xrp_ml_utils.py
start_date='2020-01-15'
end_date='2020-01-16'

#cred_path='C:\\Users\\Francesco\\Desktop\\Projects\\Cal_Hacks\\prova.json'

In [0]:
#Query and query_parameters
query1 = """
Select 
  TIMESTAMP(l.CloseTime) as `TimeStamp`,
  t.Account,t.Destination,t.Fee,t.TxnSignature,t.AmountXRP,t.LedgerIndex,t.TransactionType
FROM
  `xrpledgerdata.fullhistory.transactions`t
JOIN
  `xrpledgerdata.fullhistory.ledgers` l
  on t.LedgerIndex=l.LedgerIndex
where t.TransactionResult = "tesSUCCESS"

AND TIMESTAMP(l.CloseTime) >=TIMESTAMP(@start_date)
AND TIMESTAMP(l.CloseTime) <=TIMESTAMP(@end_date)


ORDER BY TimeStamp

LIMIT 10000000

"""
query_params1 = [
    bigquery.ScalarQueryParameter("start_date", "STRING", start_date),
    bigquery.ScalarQueryParameter("end_date", "STRING", end_date)
]

In [0]:
from google.colab import auth
auth.authenticate_user()
client = bigquery.Client(project=project_id)
#client = bigquery.Client()
client

job_config = bigquery.QueryJobConfig()
job_config.query_parameters = query_params1


In [0]:
a = client.query(query1, job_config=job_config).to_dataframe()

In [76]:
a[1:50]

,TimeStamp,Account,Destination,Fee,TxnSignature,AmountXRP,LedgerIndex,TransactionType
1,2020-01-15 00:00:00+00:00,rs9tBKt96q9gwrePKPqimUuF7vErgMaker,None,20,3045022100960FD4E0F65E39197C4127397D960F5CCDB2...,NaN,52743117,OfferCreate
2,2020-01-15 00:00:00+00:00,r4AZpDKVoBxVcYUJCWMcqZzyWsHTteC4ZE,None,12,3044022068181ACE5EDBEE28306F9804963576FA65BB37...,NaN,52743117,OfferCreate
3,2020-01-15 00:00:00+00:00,rH2k8SkwoWgwry9J89jgFP9NbSWu13jnsu,None,92,3045022100EF1EA6192A8251BB38DCF88BF02E6E7293A5...,NaN,52743117,OfferCreate
4,2020-01-15 00:00:00+00:00,rh3VLyj1GbQjX7eA15BwUagEhSrPHmLkSR,None,12,3045022100927FEB32EFDD89F34931F957D8ABF2642ED7...,NaN,52743117,OfferCreate
5,2020-01-15 00:00:00+00:00,r4dgY6Mzob3NVq8CFYdEiPnXKboRScsXRu,None,12,3045022100A788F8E882BC4CCA6FD3EF5CDE589112D7D6...,NaN,52743117,OfferCreate
6,2020-01-15 00:00:00+00:00,rJumr5e1HwiuV543H7bqixhtFreChWTaHH,None,12,3045022100F53E77FFCCAAB7F6B14CFCE7834F03F9B9E0...,NaN,52743117,OfferCreate
7,2020-01-15 00:00:00+00:00,rwchA2b36zu2r6CJfEMzPLQ1cmciKFcw9t,None,12,3045022100DF032DA3BD1A7375272E359EA395DAE23014...,NaN,52743117,OfferCreate
8,2020-01-15 00:00:00+00:00,rPsmHDMkheWZvbAkTA8A9bVnUdadPn7XBK,raQwCVAJVqjrVm1Nj5SFRcX8i22BhdC9WA,10,3045022100EF250BBB0FC8CFA495B662FF899F58FCEC2C...,3.000000e+11,52743117,Payment
9,2020-01-15 00:00:00+00:00,rH2k8SkwoWgwry9J89jgFP9NbSWu13jnsu,None,164,3045022100D500990AE3194EF42EBC62FC2EB673EC69FB...,NaN,52743117,OfferCreate
10,2020-01-15 00:00:00+00:00,rh3VLyj1GbQjX7eA15BwUagEhSrPHmLkSR,None,12,304402201E78BE216AFB414E60A26D2D401C24063D2672...,NaN,52743117,OfferCreate


In [61]:
a['TransactionType'].value_counts()

OfferCreate      694839
Payment           77610
OfferCancel       35869
AccountSet          616
TrustSet             74
SignerListSet         3
EscrowFinish          1
Name: TransactionType, dtype: int64

In [62]:
a.isnull().sum()

TimeStamp               0
Account                 0
Destination        731402
Fee                     0
TxnSignature         2732
AmountXRP          760543
LedgerIndex             0
TransactionType         0
dtype: int64

In [0]:
a[1:50]

In [75]:
a.groupby(['LedgerIndex']).count()


,TimeStamp,Account,Destination,Fee,TxnSignature,AmountXRP,TransactionType
LedgerIndex,,,,,,,
52743117,41,41,3,41,41,3,41
52743118,57,57,22,57,57,2,57
52743119,42,42,1,42,41,1,42
52743120,47,47,2,47,47,2,47
52743121,60,60,5,60,60,5,60
...,...,...,...,...,...,...,...
52765287,40,40,1,40,40,1,40
52765288,33,33,3,33,33,3,33
52765289,44,44,0,44,44,0,44


In [0]:
a.groupby(['LedgerIndex']).count()

I'm now starting to use Ravi's function to call different queries

In [0]:
def gbq_query(query, query_params=None):
    """
    Run a query against Google Big Query, returning a pandas dataframe of the result.

    Parameters
    ----------
    query: str
        The query string
    cred: obj
        Credential class instance
    
    query_params: list, optional
        The query parameters to pass into the query string
    """
    client = bigquery.Client(project=project_id)
    job_config = bigquery.QueryJobConfig()
    job_config.query_parameters = query_params
    return client.query(query, job_config=job_config).to_dataframe()

In [0]:
#Query and query_parameters
query2 = """
Select 
  TIMESTAMP(l.CloseTime) as `TimeStamp`,
  t.Account,t.Destination,t.Fee,t.TxnSignature,t.AmountXRP,t.LedgerIndex,t.TransactionType
FROM
  `xrpledgerdata.fullhistory.transactions`t
JOIN
  `xrpledgerdata.fullhistory.ledgers` l
  on t.LedgerIndex=l.LedgerIndex
where t.TransactionResult = "tesSUCCESS"
AND TxnSignature IS NOT NULL
AND Destination IS NOT NULL
AND AmountXRP  IS NOT NULL
AND TIMESTAMP(l.CloseTime) >=TIMESTAMP(@start_date)
AND TIMESTAMP(l.CloseTime) <=TIMESTAMP(@end_date)


ORDER BY TimeStamp

LIMIT 10000000

"""
query_params2 = [
    bigquery.ScalarQueryParameter("start_date", "STRING", start_date),
    bigquery.ScalarQueryParameter("end_date", "STRING", end_date)
]

In [0]:
b = gbq_query(query2,query_params1)

In [50]:
b

,TimeStamp,Account,Destination,Fee,TxnSignature,AmountXRP,LedgerIndex,TransactionType
0,2020-01-15 00:00:00+00:00,rEVrb2XPMxs3Wq1zKgFaYZers9nKCWUNd5,rDxfhNRgCDNDckm45zT5ayhKDC4Ljm7UoP,15,3045022100FF47F5970590FA693BBA1A73B5813B210BFE...,3999000000,52743117,Payment
1,2020-01-15 00:00:00+00:00,rPsmHDMkheWZvbAkTA8A9bVnUdadPn7XBK,raQwCVAJVqjrVm1Nj5SFRcX8i22BhdC9WA,10,3045022100EF250BBB0FC8CFA495B662FF899F58FCEC2C...,300000000000,52743117,Payment
2,2020-01-15 00:00:00+00:00,raQwCVAJVqjrVm1Nj5SFRcX8i22BhdC9WA,rEVrb2XPMxs3Wq1zKgFaYZers9nKCWUNd5,1000,304402201C6156E32F9D789EE91B152E1831239C98F472...,178135179070,52743117,Payment
3,2020-01-15 00:00:01+00:00,rLHtvB1kZimYJsDCqCX4iQxRtSAqqFtrA7,rEb8TK3gBgk5auZkwc6sHnwrGVJH8DuaLh,10,3045022100FFC811F41C845C96FB2434D72C7774DA7699...,4046000000,52743118,Payment
4,2020-01-15 00:00:01+00:00,rw2ciyaNshpHe7bCHo4bRWq6pqqynnWKQg,rLNaPoKeeBjZe2qs6x52yVPZpZ8td4dc6w,40,304402200DC43FCDE41BA0C9446F66A5CB2B64C659E1C4...,938193373,52743118,Payment
...,...,...,...,...,...,...,...,...
45732,2020-01-15 23:59:42+00:00,rK7LSKygRUu9y9xcuhkWbcMRKRF5HVWwVL,rU2mEJSLqBRkYLVTv55rFTgQajkLTnT6mA,100000,3045022100B5313BE0AE3CB25649AA01B095373ADEA688...,45859461,52765288,Payment
45733,2020-01-15 23:59:42+00:00,r4eEbLKZGbVSBHnSUBZW8i5XaMjGLdqT4a,r9KM11Pmc7iBQaPDFgrPMfnkUsX6cS93rp,6000,30440220191F9077A6E7C1ED0585E82DB39FE1CCAE0AC6...,1090115351,52765288,Payment
45734,2020-01-15 23:59:51+00:00,rPVMhWBsfF9iMXYj3aAzJVkPDTFNSyWdKy,rU7xJs7QmjbiyxpEozNYUFQxaRD5kueY7z,12,30440220214F0D1D0B668048CE0DC8EDA7BC817F3ED598...,534100584,52765290,Payment
45735,2020-01-15 23:59:51+00:00,r4eEbLKZGbVSBHnSUBZW8i5XaMjGLdqT4a,rEb8TK3gBgk5auZkwc6sHnwrGVJH8DuaLh,6000,30440220156AD82DC77B5731D044C301119A330F3DC8CE...,107776620000,52765290,Payment


In [51]:
b['TransactionType'].value_counts()

Payment    45737
Name: TransactionType, dtype: int64